In [1]:
from meta_transformer import transformer
import jax
from jax import random
import jax.numpy as jnp
import haiku as hk
from jax import nn

rng = random.PRNGKey(42)

2023-04-15 15:04:51.963285: E external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2023-04-15 15:04:51.963341: E external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:312] kernel version 515.86.1 does not match DSO version 515.105.1 -- cannot find working devices in this configuration
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [2]:
def forward(tokens):
    t = transformer.Classifier(
        transformer=transformer.Transformer(
            num_heads=8,
            num_layers=6,
            key_size=64,
            dropout_rate=0.1,
        ),
        model_size=512,
        num_classes=10,
    )
    return t(tokens, is_training=True)

In [3]:
model = hk.transform(forward)
chunks = random.normal(rng, (5, 30, 512))
params = model.init(rng, chunks)

In [4]:
model.apply(params, rng, chunks).shape

(5, 30, 10)

In [5]:
def loss(params, rng, tokens, labels):
    targets = jax.nn.one_hot(labels, 10)  # [B, C]
    logits = model.apply(params, rng, tokens)[:, 0, :]  # [B, C]
    assert logits.shape == targets.shape
    return jnp.sum(targets * nn.log_softmax(logits, axis=-1), axis=-1).mean()

In [6]:
loss(params, rng, chunks, jnp.array([1, 2, 3, 4, 5]))

Array(-2.747837, dtype=float32)

In [7]:
from meta_transformer.mnist import mnist

In [17]:
os.mkdir("/home/lauro/mnist-test")